In [1]:
import os
import json
import time
import copy
from datetime import datetime
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns
import lovely_tensors as lt # can be removed
import numpy as np
import dill
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from fl2o.optimizee import MLPOptee, CustomParams
from fl2o.optimizee_modules import MetaParameter
from fl2o.optimizer import GD, FGD, AFOGD, CFGD, CFGD_ClosedForm
from fl2o.l2o import L2O
from fl2o.data import MNIST, CustomTask, generate_least_squares_task
from fl2o.training import do_fit, find_best_lr, meta_train, get_optimal_lr
from fl2o.utils import plot_log, plotter

lt.monkey_patch() # can be removed

DATA_PATH = os.getenv("DATA_PATH")
CKPT_PATH = os.getenv("CKPT_PATH")
DEVICE = os.getenv("DEVICE", "cpu")

print(f"{DATA_PATH=}\n{CKPT_PATH=}\n{DEVICE=}")

DATA_PATH='/home/jsobotka/Desktop/Dev/data'
CKPT_PATH='/media/jsobotka/ext_ssd/fl2o/ckpt'
DEVICE='cuda'


In [ ]:
### load previous checkpoint (and skip meta-training of a new l2O optimizer)
ckpt = torch.load(
    os.path.join(
        CKPT_PATH,
        "l2o",
        "25-01_00-34__L2O__CFGD_ClosedForm",
        "ckpt.pth"
    ),
    map_location=torch.device(DEVICE),
    pickle_module=dill,
)
l2o_dict = ckpt["l2o_dict"]
l2o_dict_best = ckpt["l2o_dict_best"]
log = ckpt["log"]
config = ckpt["config"]
print(json.dumps(config, indent=4, default=str))

## Least Squares

$$
\begin{aligned}
    \min_{x} f(x) = \frac{1}{2} ||W^T x - y||_2^2 \\
    \text{where } W \in \mathbb{R}^{d \times m}, y \in \mathbb{R}^m
\end{aligned}
$$

In [ ]:
config = {
    "time": datetime.now().strftime("%d-%m_%H-%M"),
}

### data (task)
config["data"] = {
    "d": 100,
    "m": 100,
    "data_cls": CustomTask,
}
config["data"]["data_config"] = {
    "task": generate_least_squares_task,
    "task_config": {
        "d": config["data"]["d"],
        "m": config["data"]["m"],
        "verbose": False,
        "device": DEVICE,
    },
}


### optimizee
config["optee"] = {
    "optee_cls": CustomParams,
    "optee_config": {
        "dim": (1, config["data"]["d"]),
        "init_params": "randn",
        # "param_func": None,
    },
}

### optimizer
config["opter"] = {
    "opter_cls": L2O,
    "opter_config": {
        "in_dim": 3, # len(in_features) + 1
        "out_dim": 3,
        "hidden_sz": 40,
        "in_features": ("grad", "iter_num_enc"),
        "base_opter_cls": CFGD_ClosedForm,
        "base_opter_config": {
            "lr": get_optimal_lr,
            "gamma": None,
            "c": None,
            "version": "NA",
            "device": DEVICE,
        },
        "params_to_optimize": {
            # "gamma": {
            #     "idx": 0,
            #     "act_fns": ("identity", "diag"),
            # },
            # "gamma": {
            #     "idx": 0,
            #     "act_fns": ("alpha_to_gamma", "diag"),
            #     "beta": 0.,
            # },
            "gamma": {
                "idx": (0, 1),
                "act_fns": ("alpha_beta_to_gamma", "diag"),
            },
            "c": {
                "idx": 2,
                "act_fns": ("identity",),
            },
        },
    },
}
config["meta_training_config"] = {
    "meta_opter_cls": optim.Adam,
    "meta_opter_config": {
        "lr": 1e-3,
    },
    "n_runs": 2000,
    "unroll": 20,
    "loggers": [
        {
            "every_nth_run": 20,
            "logger_fn": partial(plotter, to_plot="gamma"),
        },
        {
            "every_nth_run": 20,
            "logger_fn": partial(plotter, to_plot="c"),
        }
    ],
}

### other
config.update({
    "n_iters": 800,
    "l2o_dict": None,
    "additional_metrics": {
        "gamma": lambda opter, **kwargs: \
            # opter.base_opter.param_groups[0]["gamma"].item() \
            opter.base_opter.param_groups[0]["gamma"].mean().item() \
            if hasattr(opter, "base_opter") else opter.param_groups[0].get("gamma", None),
        "c": lambda opter, **kwargs: \
            # opter.base_opter.param_groups[0]["c"].item() \
            opter.base_opter.param_groups[0]["c"].mean().item() \
            if hasattr(opter, "base_opter") else opter.param_groups[0].get("c", None),
        # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
        #     torch.norm(task["x_tik_solution"](gamma=1., c=1) - optee.params.detach(), p=2).item(),
        # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
        #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
    },
    "ckpt_config": {
        "ckpt_every_nth_run": 50,
        "ckpt_dir": os.path.join(
            CKPT_PATH,
            "l2o",
            config["time"] + "__"\
                + config["opter"]["opter_cls"].__name__ + "__"\
                + config["opter"]["opter_config"]["base_opter_cls"].__name__,
        ),
    },
    "device": DEVICE,
    "seed": 0,
})
config["ckpt_config"]["ckpt_dir_meta_training"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_training",
)
config["ckpt_config"]["ckpt_dir_meta_testing"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_testing",
)

### make dirs
os.makedirs(config["ckpt_config"]["ckpt_dir"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_training"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_testing"], exist_ok=True)

### save config
with open(os.path.join(config["ckpt_config"]["ckpt_dir"], "config.json"), "w") as f:
    json.dump(config, f, indent=4, default=str)

print(f"Path to checkpoints: {config['ckpt_config']['ckpt_dir']}")

In [ ]:
### meta train
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])
l2o_dict, l2o_dict_best, log = meta_train(
    opter_cls=config["opter"]["opter_cls"],
    opter_config=config["opter"]["opter_config"],
    optee_cls=config["optee"]["optee_cls"],
    optee_config=config["optee"]["optee_config"],
    data_cls=config["data"]["data_cls"],
    data_config=config["data"]["data_config"],
    n_iters=config["n_iters"],
    meta_opter_cls=config["meta_training_config"]["meta_opter_cls"],
    meta_opter_config=config["meta_training_config"]["meta_opter_config"],
    n_runs=config["meta_training_config"]["n_runs"],
    unroll=config["meta_training_config"]["unroll"],
    additional_metrics=config["additional_metrics"],
    loggers=config["meta_training_config"]["loggers"],
    ckpt_config=config["ckpt_config"],
    
    ### keep meta-training
    # l2o_dict=l2o_dict,
    # l2o_dict_best=l2o_dict_best,
    # log=log,
)

### save checkpoint
torch.save({
    "l2o_dict": l2o_dict,
    "l2o_dict_best": l2o_dict_best,
    "log": log,
    "config": config,
}, os.path.join(config["ckpt_config"]["ckpt_dir"], "ckpt.pth"), pickle_module=dill)

plt.plot(log["loss_sum"])

In [ ]:
### meta-testing config
test_d, test_m = 100, 100
n_test_runs = 1
test_run_iters = 3000
test_runs_seed = config["seed"]
# test_runs_seed = 11

runs = dict()

runs["GD"] = {
    "update_config": {
        "data": {
            "d": test_d,
            "m": test_m,
            "data_cls": CustomTask,
            "data_config": {
                "task": generate_least_squares_task,
                "task_config": {
                    "d": test_d,
                    "m": test_m,
                    "verbose": False,
                    "device": DEVICE,
                },
            },
        },
        "optee": {
            "optee_cls": CustomParams,
            "optee_config": {
                "dim": (1, test_d),
                "init_params": "randn",
                # "param_func": None,
            },
        },
        "opter": {
            "opter_cls": GD,
            "opter_config": {
                "lr": get_optimal_lr,
                "device": DEVICE,
            },
        },
        "additional_metrics": {
            # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
            # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
            # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
            "cos_sim(d, x.grad)": lambda opter, **kwargs: \
                torch.cosine_similarity(
                    opter.param_groups[0]["last_update"].flatten(),
                    opter.param_groups[0]["last_grad"].flatten(),
                    dim=0
                ).item(),
            "last_lr": lambda opter, **kwargs: \
                opter.param_groups[0]["last_lr"].item() if type(opter.param_groups[0]["last_lr"]) == torch.Tensor else opter.param_groups[0]["last_lr"],
        },
        "n_iters": test_run_iters,
    },
    "plot_config": {
        "color": "black",
        "linestyle": "dashed",
    },
}

# for gamma in [-0.23, 0.01, 0.05, 0.5, 1.]:
#     runs[r"NA-CFGD_ClosedForm, $\gamma$=" + str(gamma)] = {
#         "update_config": {
#             "data": {
#                 "d": test_d,
#                 "m": test_m,
#                 "data_cls": CustomTask,
#                 "data_config": {
#                     "task": generate_least_squares_task,
#                     "task_config": {
#                         "d": test_d,
#                         "m": test_m,
#                         "verbose": False,
#                         "device": DEVICE,
#                     },
#                 },
#             },
#             "optee": {
#                 "optee_cls": CustomParams,
#                 "optee_config": {
#                     "dim": (1, test_d),
#                     "init_params": "randn",
#                     # "param_func": None,
#                 },
#             },
#             "opter": {
#                 "opter_cls": CFGD_ClosedForm,
#                 "opter_config": {
#                     "lr": get_optimal_lr,
#                     "gamma": gamma,
#                     "c": 1.,
#                     "device": DEVICE,
#                 },
#             },
#             "additional_metrics": {
#                 # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
#                 #     torch.norm(task["x_tik_solution"](gamma=gamma, c=1) - optee.params.detach(), p=2).item(),
#                 # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
#                 #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
#                 # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
#                 "cos_sim(d, x.grad)": lambda opter, **kwargs: \
#                     torch.cosine_similarity(
#                         opter.state[0]["last_update"].flatten(),
#                         opter.state[0]["last_grad"].flatten(),
#                         dim=0
#                     ).item(),
#                 "last_lr": lambda opter, **kwargs: \
#                     opter.state[0]["last_lr"].item() if type(opter.state[0]["last_lr"]) == torch.Tensor else opter.state[0]["last_lr"],
#             },
#             "n_iters": test_run_iters,
#         },
#         "plot_config": {
#             "linestyle": "dashed",
#         },
#     }

runs["L2O + CFGD_ClosedForm"] = {
    "update_config": {
        "data": {
            "d": test_d,
            "m": test_m,
            "data_cls": CustomTask,
            "data_config": {
                "task": generate_least_squares_task,
                "task_config": {
                    "d": test_d,
                    "m": test_m,
                    "verbose": False,
                    "device": DEVICE,
                },
            },
        },
        "optee": {
            "optee_cls": CustomParams,
            "optee_config": {
                "dim": (1, test_d),
                "init_params": "randn",
                # "param_func": None,
            },
        },
        "additional_metrics": {
            "gamma": lambda opter, **kwargs: \
                # opter.base_opter.param_groups[0]["gamma"].item() \
                opter.base_opter.param_groups[0]["gamma"].detach().cpu().numpy() \
                if hasattr(opter, "base_opter") else opter.param_groups[0].get("gamma", None),
            "c": lambda opter, **kwargs: \
                # opter.base_opter.param_groups[0]["c"].item() \
                opter.base_opter.param_groups[0]["c"].detach().cpu().numpy() \
                if hasattr(opter, "base_opter") else opter.param_groups[0].get("c", None),
            "alpha": lambda opter, **kwargs: \
                # opter.base_opter.param_groups[0]["gamma"].item() \
                opter.base_opter.param_groups[0]["alpha"] \
                if hasattr(opter, "base_opter") else opter.param_groups[0].get("alpha", None),
            "beta": lambda opter, **kwargs: \
                # opter.base_opter.param_groups[0]["c"].item() \
                opter.base_opter.param_groups[0]["beta"] \
                if hasattr(opter, "base_opter") else opter.param_groups[0].get("beta", None),
            # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
            # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
            # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
        },
        "l2o_dict": l2o_dict,
        # "l2o_dict": l2o_dict_best["best_l2o_dict"],
        "n_iters": test_run_iters,
    },
    "plot_config": {
        "color": "orange",
        "linewidth": "3",
    },
}

# runs["CFGD"] = {
#     "update_config": {
#         "data": {
#             "d": test_d,
#             "m": test_m,
#             "data_cls": CustomTask,
#             "data_config": {
#                 "task": generate_least_squares_task,
#                 "task_config": {
#                     "d": test_d,
#                     "m": test_m,
#                     "verbose": False,
#                     "device": DEVICE,
#                 },
#             },
#         },
#         "optee": {
#             "optee_cls": CustomParams,
#             "optee_config": {
#                 "dim": (1, test_d),
#                 "init_params": "randn",
#                 # "param_func": None,
#             },
#         },
#         "opter": {
#             "opter_cls": CFGD,
#             "opter_config": {
#                 "lr": get_optimal_lr,
#                 # "lr": 0.1,
#                 "alpha": 0.9,
#                 # "beta": 30.23,
#                 "beta": 0.,
#                 "c": 1,
#                 "s": 3,
#                 "device": DEVICE,
#             },
#         },
#         "additional_metrics": {
#             # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
#             # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
#             # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
#             # "cos_sim(d, x.grad)": lambda opter, **kwargs: \
#             #     torch.cosine_similarity(
#             #         opter.state[0]["last_update"].flatten(),
#             #         opter.state[0]["last_grad"].flatten(),
#             #         dim=0
#             #     ).item(),
#             # "last_lr": lambda opter, **kwargs: \
#             #     opter.state[0]["last_lr"].item() if type(opter.state[0]["last_lr"]) == torch.Tensor else opter.state[0]["last_lr"],
#         },
#         "n_iters": test_run_iters,
#     },
#     "plot_config": {
#         "color": "orange",
#         "linewidth": "3",
#     },
# }

In [ ]:
### run all
for run_name in runs.keys():
    if "log" in runs[run_name]:
        continue # already run
    run_config = copy.deepcopy(config)
    if "update_config" in runs[run_name] and runs[run_name]["update_config"] is not None:
        run_config.update(runs[run_name]["update_config"])
    print(f"{run_name}:")
    
    torch.manual_seed(test_runs_seed)
    np.random.seed(test_runs_seed)

    if "lr" in run_config["opter"]["opter_config"] \
        and run_config["opter"]["opter_config"]["lr"] == find_best_lr:
        print("  > Finding best lr...")
        run_config["opter"]["opter_config"]["lr"] = find_best_lr(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            # loss_fn=run_config["loss_fn"],
            n_iters=120,
            n_tests=1,
            consider_metric="loss",
            lrs_to_try=[0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
        )
        print(f"  > Best lr: {run_config['opter']['opter_config']['lr']}")

    print("  > Running...")
    runs[run_name]["log"] = dict()
    for i in range(n_test_runs):
        print(f"    > Run {i+1}/{n_test_runs}...")

        ### check if L2O has been meta-trained
        assert not run_config["opter"]["opter_cls"] == L2O or run_config["l2o_dict"] is not None

        curr_log = do_fit(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            n_iters=run_config["n_iters"],
            l2o_dict=run_config["l2o_dict"],
            in_meta_training=False,
            additional_metrics=run_config["additional_metrics"],
        )[0]

        for metric_name in curr_log.keys():
            if metric_name not in runs[run_name]["log"]:
                runs[run_name]["log"][metric_name] = []
            runs[run_name]["log"][metric_name].append(curr_log[metric_name])

    runs[run_name]["config"] = run_config

In [ ]:
plot_log(
    runs,
    only_metrics=["loss"],
    log_metrics=["loss", "l2_dist(x_tik*, x)", "l2_dist(x*, x)"],
    conv_win=1,
    min_max_y_config={
        "last_lr": (0, 100),
    },
    save_to=os.path.join(
        config["ckpt_config"]["ckpt_dir"],
        f"loss_l2o_best_dict_{test_d}d_{test_m}m_{n_test_runs}runs_{test_run_iters}iters.png"
    ),
)

In [ ]:
x = torch.randn(10, 30)
w = torch.randn(3, 10, 30, requires_grad=True)
b = torch.randn(3, 10, requires_grad=True)

In [ ]:
l = torch.nn.Linear(30, 10, bias=True)
l.weight.data = x
l.weight.requires_grad_(False)
l.bias.data = b.T
l.bias.requires_grad_(True)

In [ ]:
l = torch.nn.Linear(40, 10, bias=False)
l.weight.data[:, :x.shape[1]] = x
l.weight.data[:, x.shape[1]:] = 0.
for b_idx in range(b.shape[-1]):
    l.weight.data[b_idx, x.shape[1] + b_idx] = 1.
l.weight.requires_grad_(False)
# l.bias.data = b
# l.bias.requires_grad_(True)

### grad grad wrt w and b
inp = torch.cat([w, b.unsqueeze(1).expand(-1, w.shape[1], -1)], dim=2)
w_grad, b_grad = torch.autograd.grad(l(inp).sum(), (w, b))
b_grad /= w.shape[1]  # for expanding b
# l(inp).sum().backward()
# w_grad = w.grad
# b_grad = b.grad / w.shape[1]

In [ ]:
optee = MLPOptee().to(DEVICE)

In [ ]:
d = MNIST(device=DEVICE)

In [ ]:
task = d.sample()

In [ ]:
from fl2o.optimizee import get_mlp_optee_mirror

model = get_mlp_optee_mirror(optee)

In [ ]:
import autograd_hacks

autograd_hacks.add_hooks(model)
autograd_hacks.backprop_hess(model(task), hess_type='CrossEntropy')
autograd_hacks.compute_hess(model)
for param in model.parameters():
    print(param.hess)  # print Hessian of param

In [ ]:
for p_idx, (n, p) in enumerate(optee.all_named_parameters()):
    if p_idx == 0:
        continue
    deriv_eval_points = torch.rand_like(p.data)
    pp = p.data.detach().clone().view(-1).unsqueeze(0).repeat(p.shape.numel(), 1) # (param_size, param_size)
    pp[torch.arange(p.shape.numel()), torch.arange(p.shape.numel())] = deriv_eval_points.view(-1) # (param_size, param_size)
    
    ### get the hessian
    forward_w_params = pp.view(p.shape.numel(), *p.shape)
    y_hat = optee.forward_w_params(task=task, params=forward_w_params, params_for=n)
    loss = task["loss_fn"](y_hat=y_hat)
    fo_out = torch.autograd.grad(
        outputs=loss,
        inputs=forward_w_params,
        create_graph=True,
    )[0]
    ### get he hessian
    he_out = torch.autograd.grad(
        outputs=fo_out,
        inputs=forward_w_params,
        # grad_outputs=torch.ones_like(fo_out),
    )[0]

In [ ]:
forward_w_params = [p.detach().clone().view(-1) for p in optee.parameters()]
forward_w_params = torch.stack(forward_w_params)

In [ ]:
y_hat = optee.forward_w_params(task=task, params=forward_w_params)
loss = task["loss_fn"](y_hat=y_hat)
fo_out = torch.autograd.grad(
    outputs=loss,
    inputs=forward_w_params,
)[0]

#### Analyze strategy

In [ ]:
# gammas = np.stack(runs["L2O + CFGD_ClosedForm"]["log"]["gamma"])
# gammas = gammas.T.diagonal().transpose(1, 0, 2)  # (n_test_runs, n_iters, D)

alphas = np.stack(runs["L2O + CFGD_ClosedForm"]["log"]["alpha"])
# alphas = alphas.T.diagonal().transpose(1, 0, 2)  # (n_test_runs, n_iters, D)

betas = np.stack(runs["L2O + CFGD_ClosedForm"]["log"]["beta"])
# betas = betas.T.diagonal().transpose(1, 0, 2)  # (n_test_runs, n_iters, D)

cs = np.stack(runs["L2O + CFGD_ClosedForm"]["log"]["c"])  # (n_test_runs, n_iters, D)

In [ ]:
test_run_idx = 0
to_plot_name = "c"
latex_expr = "c"

fig = plt.figure(figsize=(15, 6), facecolor="w")
fig.suptitle(rf"L2O-CFGD: ${latex_expr}$", fontsize=16)

ax = fig.add_subplot(121)
if to_plot_name == "alpha":
    to_plot = alphas
elif to_plot_name == "beta":
    to_plot = betas
elif to_plot_name == "gamma":
    to_plot = gammas
elif to_plot_name == "c":
    to_plot = cs
else:
    raise ValueError(f"Unknown to_plot: {to_plot_name}")

plt.plot(to_plot[test_run_idx].squeeze(), alpha=0.1, color="grey")
plt.plot(to_plot[test_run_idx].squeeze().mean(-1), color="orange", linewidth=3)
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(rf"${latex_expr}$", fontsize=13)

### share y-axis with left plot
ax = fig.add_subplot(122, sharey=ax)
plt.plot(to_plot[test_run_idx].squeeze()[:,:3])
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(rf"${latex_expr}$", fontsize=13)
ax.legend(["Parameter #1", "Parameter #2", "Parameter #3"])

plt.tight_layout(h_pad=2.5)
save_to = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    f"strategy_{to_plot_name}_{test_d}d_{test_m}m_{n_test_runs}runs_{test_run_iters}iters.png"
)
fig.savefig(save_to)

plt.show()

In [ ]:
test_run_idx = 0

fig = plt.figure(figsize=(15, 6))
fig.suptitle(r"L2O-CFGD: $\gamma$", fontsize=16)

ax = fig.add_subplot(121)
plt.plot(gammas[test_run_idx], alpha=0.1, color="grey")
plt.plot(gammas[test_run_idx].mean(-1), color="orange", linewidth=3)
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(r"$\gamma$", fontsize=13)

### share y-axis with left plot
ax = fig.add_subplot(122, sharey=ax)
plt.plot(gammas[test_run_idx,:,:3])
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(r"$\gamma$", fontsize=13)
ax.legend(["Parameter #1", "Parameter #2", "Parameter #3"])

plt.show()

In [ ]:
test_run_idx = 0

fig = plt.figure(figsize=(15, 6))
fig.suptitle(r"L2O-CFGD: $c$", fontsize=16)

ax = fig.add_subplot(121)
plt.plot(cs[test_run_idx], alpha=0.1, color="grey")
plt.plot(cs[test_run_idx].mean(-1), color="orange", linewidth=3)
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(r"$c$", fontsize=13)

### share y-axis with left plot
ax = fig.add_subplot(122, sharey=ax)
plt.plot(cs[test_run_idx,:,:3])
ax.set_xlabel("Iteration", fontsize=13)
ax.set_ylabel(r"$c$", fontsize=13)
ax.legend(["Parameter #1", "Parameter #2", "Parameter #3"])

plt.show()

## Quadratic Objective Function (Figure 1)

In [ ]:
"""
Quadratic objective function
    min_x f(x, y) = 10 * x^2 + y^2
"""
def task_gen(device="cpu"):
    ### Least squares problem 1/2 x^T A x + b^T x
    A = torch.tensor([[10., 0.], [0., 1.]], device=device)
    b = torch.tensor([[0., 0.]], device=device).T
    x_solution = torch.tensor([0., 0.], device=device)

    loss_fn = lambda y_hat: 0.5 * y_hat @ A @ y_hat.T + b.T @ y_hat.T

    return {
        "A": A,
        "b": b,
        "loss_fn": loss_fn,
        "x_solution": x_solution,
    }

In [ ]:
config = {
    "optee": {
        "optee_cls": CustomParams,
        "optee_config": {
            "dim": (1, 2),
            "init_params": torch.tensor([[1., -10.]], device=DEVICE),
            "param_func": None,
        },
    },
    "opter": {
        "opter_cls": CFGD_ClosedForm,
        "opter_config": {
            "lr": get_optimal_lr,
            "gamma": -1.,
            "c": None,
            "version": "AT",
            "init_points": [torch.tensor([-1., -1.], device=DEVICE)],
            "device": DEVICE,
        },
    },
    "data": {
        "data_cls": CustomTask,
        "data_config": {
            "task": partial(task_gen, verbose=False, device=DEVICE),
            "device": DEVICE,
        },
    },
    "n_iters": 50,
    "l2o_dict": None,
    "additional_metrics": {
        "l2_dist(x*, x)": lambda task, optee, **kwargs: \
            torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
    },
    "device": DEVICE,
    "seed": 0,
}

In [ ]:
### runs config
runs = dict()

runs["GD"] = {
    "update_config": {
        "opter": {
            "opter_cls": GD,
            "opter_config": {
                "lr": get_optimal_lr,
                "device": DEVICE,
            },
        },
        "additional_metrics": {
            "l2_dist(x*, x)": lambda task, optee, **kwargs: \
                torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
            "x": lambda optee, **kwargs: optee.params.detach().cpu().numpy(),
        },
    },
    "plot_config": {
        "color": "black",
        "linestyle": "dashed",
    },
}

for gamma in [-1., 0.01, 0.05, 0.25, 0.5, 1., 10.]:
    runs[r"CFGD_ClosedForm, $\gamma$=" + str(gamma)] = {
        "update_config": {
            "opter": {
                "opter_cls": CFGD_ClosedForm,
                "opter_config": {
                    "lr": get_optimal_lr,
                    "gamma": gamma,
                    "c": None,
                    "version": "AT",
                    "init_points": [torch.tensor([-1., -1.], device=DEVICE)],
                    "device": DEVICE,
                },
            },
            "additional_metrics": {
                "l2_dist(x*, x)": lambda task, optee, **kwargs: \
                    torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
                "x": lambda optee, **kwargs: optee.params.detach().cpu().numpy(),
            },
        },
        # "plot_config": {
        #     "linestyle": "dashed",
        # },
    }

In [ ]:
### run all
for run_name in runs.keys():
    if "log" in runs[run_name]:
        continue # already run
    run_config = copy.deepcopy(config)
    if "update_config" in runs[run_name] and runs[run_name]["update_config"] is not None:
        run_config.update(runs[run_name]["update_config"])
    print(f"{run_name}:")
    
    torch.manual_seed(config["seed"])
    np.random.seed(config["seed"])

    if "lr" in run_config["opter"]["opter_config"] \
        and run_config["opter"]["opter_config"]["lr"] == find_best_lr:
        print("  > Finding best lr...")
        run_config["opter"]["opter_config"]["lr"] = find_best_lr(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            # loss_fn=run_config["loss_fn"],
            n_iters=120,
            n_tests=1,
            consider_metric="loss",
            lrs_to_try=[0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
        )
        print(f"  > Best lr: {run_config['opter']['opter_config']['lr']}")

    print("  > Running...")
    runs[run_name]["log"], _, _ = do_fit(
        opter_cls=run_config["opter"]["opter_cls"],
        opter_config=run_config["opter"]["opter_config"],
        optee_cls=run_config["optee"]["optee_cls"],
        optee_config=run_config["optee"]["optee_config"],
        data_cls=run_config["data"]["data_cls"],
        data_config=run_config["data"]["data_config"],
        n_iters=run_config["n_iters"],
        l2o_dict=run_config["l2o_dict"],
        in_meta_training=False,
        additional_metrics=run_config["additional_metrics"],
    )
    runs[run_name]["config"] = run_config

In [ ]:
plot_log(
    runs,
    only_metrics=["loss", "l2_dist(x*, x)", "time"],
    log_metrics=["loss", "l2_dist(x_tik*, x)", "l2_dist(x*, x)"],
    conv_win=1,
)

In [ ]:
def objective_function(x, y):
    return 10 * x**2 + y**2

def plot_optimizer_steps(runs, num_steps=10, only_opters=None, starting_points=None):
    plt.figure(figsize=(15, 8))

    # Plot the contour of the objective function
    x_range = np.linspace(-1.5, 1.5, 100)
    y_range = np.linspace(-10.5, 2, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = objective_function(X, Y)
    contour = plt.contour(Y, X, Z, levels=20, cmap='viridis')
    plt.colorbar(contour, label='Objective Function Value')

    for optimizer, data in runs.items():
        if only_opters is not None and optimizer not in only_opters:
            continue


        x_values = [x.flatten() for x in data["log"]["x"][:num_steps]]
        if starting_points is not None:
            x_values = starting_points + x_values
        x_values = np.stack(x_values)

        plt.plot(x_values[:, 1], x_values[:, 0], label=optimizer, marker='o', markersize=8, linewidth=3)

    plt.title('Optimizer Steps')
    plt.xlabel('y')
    plt.ylabel('x')
    plt.legend()
    plt.show()

# Example usage:
# Replace 'runs' with your actual dictionary
# For demonstration, I'm using a simplified structure
# runs = {
#     "GD": {"log": {"x": [torch.randn(1, 2) for _ in range(20)]}},
#     "Adam": {"log": {"x": [torch.randn(1, 2) for _ in range(20)]}},
#     # Add more optimizers as needed
# }

plot_optimizer_steps(
    runs,
    only_opters=[
        "GD",
        "CFGD_ClosedForm, $\\gamma$=-1.0",
        "CFGD_ClosedForm, $\\gamma$=0.5",
        "CFGD_ClosedForm, $\\gamma$=10.0"
    ],
    num_steps=5,
    starting_points=[[1., -10.]],
)

## MNIST

In [2]:
config = {
    "time": datetime.now().strftime("%d-%m_%H-%M"),
}

### data (task)
config["data"] = {
    "data_cls": MNIST,
}
config["data"]["data_config"] = {
    "device": DEVICE,
    "preload": True,
}

### optimizee
config["optee"] = {
    "optee_cls": MLPOptee,
    "optee_config": {
        "layer_sizes": [20],
        "act_fn": nn.ReLU(),
    },
}

### optimizer
config["opter"] = {
    "opter_cls": L2O,
    "opter_config": {
        "in_dim": 3, # len(in_features) + 1
        "out_dim": 3,
        "hidden_sz": 40,
        "in_features": ("grad", "iter_num_enc"),
        "base_opter_cls": CFGD,
        "base_opter_config": {
            "lr": 0.05,
            "alpha": None,
            "beta": None,
            "c": None,
            "s": 1,
            "device": DEVICE,
        },
        "params_to_optimize": {
            "alpha": {
                "idx": 0,
                "act_fns": ("sigmoid",),
            },
            "beta": {
                "idx": 1,
                "act_fns": ("identity",),
            },
            "c": {
                "idx": 2,
                "act_fns": ("identity",),
            },
        },
    },
}
config["meta_training_config"] = {
    "meta_opter_cls": optim.Adam,
    "meta_opter_config": {
        "lr": 3e-4,
    },
    "n_runs": 500,
    "unroll": 30,
    "loggers": [
        # {
        #     "every_nth_run": 20,
        #     "logger_fn": partial(plotter, to_plot="gamma"),
        # },
        # {
        #     "every_nth_run": 20,
        #     "logger_fn": partial(plotter, to_plot="c"),
        # }
    ],
}

### other
config.update({
    "n_iters": 200,
    "l2o_dict": None,
    "additional_metrics": {
        # "gamma": lambda opter, **kwargs: \
        #     # opter.base_opter.param_groups[0]["gamma"].item() \
        #     opter.base_opter.param_groups[0]["gamma"].mean().item() \
        #     if hasattr(opter, "base_opter") else opter.param_groups[0].get("gamma", None),
        # "c": lambda opter, **kwargs: \
        #     # opter.base_opter.param_groups[0]["c"].item() \
        #     opter.base_opter.param_groups[0]["c"].mean().item() \
        #     if hasattr(opter, "base_opter") else opter.param_groups[0].get("c", None),
        # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
        #     torch.norm(task["x_tik_solution"](gamma=1., c=1) - optee.params.detach(), p=2).item(),
        # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
        #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
    },
    "ckpt_config": {
        "ckpt_every_nth_run": 20,
        "ckpt_dir": os.path.join(
            CKPT_PATH,
            "l2o",
            config["time"] + "__"\
                + config["opter"]["opter_cls"].__name__ + "__"\
                + config["opter"]["opter_config"]["base_opter_cls"].__name__,
        ),
    },
    "device": DEVICE,
    "seed": 0,
})
config["ckpt_config"]["ckpt_dir_meta_training"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_training",
)
config["ckpt_config"]["ckpt_dir_meta_testing"] = os.path.join(
    config["ckpt_config"]["ckpt_dir"],
    "meta_testing",
)

### make dirs
os.makedirs(config["ckpt_config"]["ckpt_dir"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_training"], exist_ok=True)
os.makedirs(config["ckpt_config"]["ckpt_dir_meta_testing"], exist_ok=True)

### save config
with open(os.path.join(config["ckpt_config"]["ckpt_dir"], "config.json"), "w") as f:
    json.dump(config, f, indent=4, default=str)

print(f"Path to checkpoints: {config['ckpt_config']['ckpt_dir']}")

Path to checkpoints: /media/jsobotka/ext_ssd/fl2o/ckpt/l2o/30-01_16-30__L2O__CFGD


In [4]:
### meta train
torch.manual_seed(config["seed"])
np.random.seed(config["seed"])
l2o_dict, l2o_dict_best, log = meta_train(
    opter_cls=config["opter"]["opter_cls"],
    opter_config=config["opter"]["opter_config"],
    optee_cls=config["optee"]["optee_cls"],
    optee_config=config["optee"]["optee_config"],
    data_cls=config["data"]["data_cls"],
    data_config=config["data"]["data_config"],
    n_iters=config["n_iters"],
    meta_opter_cls=config["meta_training_config"]["meta_opter_cls"],
    meta_opter_config=config["meta_training_config"]["meta_opter_config"],
    n_runs=config["meta_training_config"]["n_runs"],
    unroll=config["meta_training_config"]["unroll"],
    additional_metrics=config["additional_metrics"],
    loggers=config["meta_training_config"]["loggers"],
    ckpt_config=config["ckpt_config"],
    
    ### keep meta-training
    # l2o_dict=l2o_dict,
    # l2o_dict_best=l2o_dict_best,
    # log=log,
)

### save checkpoint
torch.save({
    "l2o_dict": l2o_dict,
    "l2o_dict_best": l2o_dict_best,
    "log": log,
    "config": config,
}, os.path.join(config["ckpt_config"]["ckpt_dir"], "ckpt.pth"), pickle_module=dill)

plt.plot(log["loss_sum"])

[INFO] Starting meta-training.
[INFO] Meta-training starts.
  [1/500]> sum(loss): 100.441  last(loss): 0.281
       > new best loss sum: 100.441
  [2/500]> sum(loss): 95.766  last(loss): 0.330
       > new best loss sum: 95.766
  [3/500]> sum(loss): 95.826  last(loss): 0.315
  [4/500]> sum(loss): 95.955  last(loss): 0.279
  [5/500]> sum(loss): 99.787  last(loss): 0.266
  [6/500]> sum(loss): 94.588  last(loss): 0.244
       > new best loss sum: 94.588
  [7/500]> sum(loss): 90.156  last(loss): 0.298
       > new best loss sum: 90.156
  [8/500]> sum(loss): 95.510  last(loss): 0.348
  [9/500]> sum(loss): 98.657  last(loss): 0.374
  [10/500]> sum(loss): 97.637  last(loss): 0.278
  [11/500]> sum(loss): 96.061  last(loss): 0.297
  [12/500]> sum(loss): 91.102  last(loss): 0.306
  [13/500]> sum(loss): 95.240  last(loss): 0.360
  [14/500]> sum(loss): 95.306  last(loss): 0.343
  [15/500]> sum(loss): 95.230  last(loss): 0.322
  [16/500]> sum(loss): 95.405  last(loss): 0.308
  [17/500]> sum(loss): 

KeyboardInterrupt: 

In [ ]:
### meta-testing config
n_test_runs = 1
test_run_iters = 200
test_runs_seed = config["seed"]

runs = dict()

# runs["GD"] = {
#     "update_config": {
#         "opter": {
#             "opter_cls": GD,
#             "opter_config": {
#                 "lr": 0.3,
#                 "device": DEVICE,
#             },
#         },
#         "additional_metrics": {
#             # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
#             # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
#             # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
#             # "cos_sim(d, x.grad)": lambda opter, **kwargs: \
#             #     torch.cosine_similarity(
#             #         opter.param_groups[0]["last_update"].flatten(),
#             #         opter.param_groups[0]["last_grad"].flatten(),
#             #         dim=0
#             #     ).item(),
#             # "last_lr": lambda opter, **kwargs: \
#             #     opter.param_groups[0]["last_lr"].item() if type(opter.param_groups[0]["last_lr"]) == torch.Tensor else opter.param_groups[0]["last_lr"],
#         },
#         "n_iters": test_run_iters,
#     },
#     "plot_config": {
#         "color": "black",
#         "linestyle": "dashed",
#     },
# }

# for gamma in [-0.23, 0.01, 0.05, 0.5, 1.]:
#     runs[r"NA-CFGD_ClosedForm, $\gamma$=" + str(gamma)] = {
#         "update_config": {
#             "data": {
#                 "d": test_d,
#                 "m": test_m,
#                 "data_cls": CustomTask,
#                 "data_config": {
#                     "task": generate_least_squares_task,
#                     "task_config": {
#                         "d": test_d,
#                         "m": test_m,
#                         "verbose": False,
#                         "device": DEVICE,
#                     },
#                 },
#             },
#             "optee": {
#                 "optee_cls": CustomParams,
#                 "optee_config": {
#                     "dim": (1, test_d),
#                     "init_params": "randn",
#                     # "param_func": None,
#                 },
#             },
#             "opter": {
#                 "opter_cls": CFGD_ClosedForm,
#                 "opter_config": {
#                     "lr": get_optimal_lr,
#                     "gamma": gamma,
#                     "c": 1.,
#                     "device": DEVICE,
#                 },
#             },
#             "additional_metrics": {
#                 # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
#                 #     torch.norm(task["x_tik_solution"](gamma=gamma, c=1) - optee.params.detach(), p=2).item(),
#                 # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
#                 #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
#                 # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
#                 "cos_sim(d, x.grad)": lambda opter, **kwargs: \
#                     torch.cosine_similarity(
#                         opter.state[0]["last_update"].flatten(),
#                         opter.state[0]["last_grad"].flatten(),
#                         dim=0
#                     ).item(),
#                 "last_lr": lambda opter, **kwargs: \
#                     opter.state[0]["last_lr"].item() if type(opter.state[0]["last_lr"]) == torch.Tensor else opter.state[0]["last_lr"],
#             },
#             "n_iters": test_run_iters,
#         },
#         "plot_config": {
#             "linestyle": "dashed",
#         },
#     }

# runs["L2O + CFGD_ClosedForm"] = {
#     "update_config": {
#         "additional_metrics": {
#             "gamma": lambda opter, **kwargs: \
#                 # opter.base_opter.param_groups[0]["gamma"].item() \
#                 opter.base_opter.param_groups[0]["gamma"].detach().cpu().numpy() \
#                 if hasattr(opter, "base_opter") else opter.param_groups[0].get("gamma", None),
#             "c": lambda opter, **kwargs: \
#                 # opter.base_opter.param_groups[0]["c"].item() \
#                 opter.base_opter.param_groups[0]["c"].detach().cpu().numpy() \
#                 if hasattr(opter, "base_opter") else opter.param_groups[0].get("c", None),
#             "alpha": lambda opter, **kwargs: \
#                 # opter.base_opter.param_groups[0]["gamma"].item() \
#                 opter.base_opter.param_groups[0]["alpha"] \
#                 if hasattr(opter, "base_opter") else opter.param_groups[0].get("alpha", None),
#             "beta": lambda opter, **kwargs: \
#                 # opter.base_opter.param_groups[0]["c"].item() \
#                 opter.base_opter.param_groups[0]["beta"] \
#                 if hasattr(opter, "base_opter") else opter.param_groups[0].get("beta", None),
#             # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
#             # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
#             #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
#             # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
#         },
#         "l2o_dict": l2o_dict,
#         # "l2o_dict": l2o_dict_best["best_l2o_dict"],
#         "n_iters": test_run_iters,
#     },
#     "plot_config": {
#         "color": "orange",
#         "linewidth": "3",
#     },
# }

runs["CFGD"] = {
    "update_config": {
        "opter": {
            "opter_cls": CFGD,
            "opter_config": {
                "lr": 0.3,
                "alpha": 0.9,
                "beta": 0.2,
                "c": 1,
                "s": 1,
                "device": DEVICE,
            },
        },
        "additional_metrics": {
            # "l2_dist(x_tik*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_tik_solution"](gamma=0., c=1) - optee.params.detach(), p=2).item(),
            # "l2_dist(x*, x)": lambda task, optee, **kwargs: \
            #     torch.norm(task["x_solution"] - optee.params.detach(), p=2).item(),
            # "x": lambda y_hat, y, optee: optee.params.detach().cpu().numpy(),
            # "cos_sim(d, x.grad)": lambda opter, **kwargs: \
            #     torch.cosine_similarity(
            #         opter.state[0]["last_update"].flatten(),
            #         opter.state[0]["last_grad"].flatten(),
            #         dim=0
            #     ).item(),
            # "last_lr": lambda opter, **kwargs: \
            #     opter.state[0]["last_lr"].item() if type(opter.state[0]["last_lr"]) == torch.Tensor else opter.state[0]["last_lr"],
        },
        "n_iters": test_run_iters,
    },
    "plot_config": {
        "color": "orange",
        "linewidth": "3",
    },
}

In [ ]:
### run all
for run_name in runs.keys():
    if "log" in runs[run_name]:
        continue # already run
    run_config = copy.deepcopy(config)
    if "update_config" in runs[run_name] and runs[run_name]["update_config"] is not None:
        run_config.update(runs[run_name]["update_config"])
    print(f"{run_name}:")
    
    torch.manual_seed(test_runs_seed)
    np.random.seed(test_runs_seed)

    if "lr" in run_config["opter"]["opter_config"] \
        and run_config["opter"]["opter_config"]["lr"] == find_best_lr:
        print("  > Finding best lr...")
        run_config["opter"]["opter_config"]["lr"] = find_best_lr(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            # loss_fn=run_config["loss_fn"],
            n_iters=120,
            n_tests=1,
            consider_metric="loss",
            lrs_to_try=[0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0, 1.5, 2.0],
        )
        print(f"  > Best lr: {run_config['opter']['opter_config']['lr']}")

    print("  > Running...")
    runs[run_name]["log"] = dict()
    for i in range(n_test_runs):
        print(f"    > Run {i+1}/{n_test_runs}...")

        ### check if L2O has been meta-trained
        assert not run_config["opter"]["opter_cls"] == L2O or run_config["l2o_dict"] is not None

        curr_log = do_fit(
            opter_cls=run_config["opter"]["opter_cls"],
            opter_config=run_config["opter"]["opter_config"],
            optee_cls=run_config["optee"]["optee_cls"],
            optee_config=run_config["optee"]["optee_config"],
            data_cls=run_config["data"]["data_cls"],
            data_config=run_config["data"]["data_config"],
            n_iters=run_config["n_iters"],
            l2o_dict=run_config["l2o_dict"],
            in_meta_training=False,
            additional_metrics=run_config["additional_metrics"],
        )[0]

        for metric_name in curr_log.keys():
            if metric_name not in runs[run_name]["log"]:
                runs[run_name]["log"][metric_name] = []
            runs[run_name]["log"][metric_name].append(curr_log[metric_name])

    runs[run_name]["config"] = run_config